In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit

Cleaning Data

In [2]:
df = pd.read_csv('Most Streamed Spotify Songs 2024.csv', encoding = 'latin1')
'''num_norm = ['All Time Rank','Spotify Streams','Spotify Playlist Count','Spotify Playlist Reach','Spotify Popularity','YouTube Views','YouTube Likes','TikTok Posts','TikTok Likes','TikTok Views','Shazam Counts','Explicit Track']'''
num_norm = ['Spotify Streams', 'YouTube Views','YouTube Likes','TikTok Posts','TikTok Likes','TikTok Views','Shazam Counts']
for x in num_norm:
    df = df.dropna(subset=[x])
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3094 entries, 0 to 4599
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Track                       3094 non-null   object 
 1   Album Name                  3094 non-null   object 
 2   Artist                      3094 non-null   object 
 3   Release Date                3094 non-null   object 
 4   ISRC                        3094 non-null   object 
 5   All Time Rank               3094 non-null   object 
 6   Track Score                 3094 non-null   float64
 7   Spotify Streams             3094 non-null   object 
 8   Spotify Playlist Count      3094 non-null   object 
 9   Spotify Playlist Reach      3094 non-null   object 
 10  Spotify Popularity          2917 non-null   float64
 11  YouTube Views               3094 non-null   object 
 12  YouTube Likes               3094 non-null   object 
 13  TikTok Posts                3094 non-n

In [3]:
#Track,Artist,Release Date,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,TikTok Views,Shazam Counts,Explicit Track
columns = ['Album Name','YouTube Playlist Reach','Apple Music Playlist Count','AirPlay Spins','SiriusXM Spins','Deezer Playlist Count','Deezer Playlist Reach','Amazon Playlist Count','Pandora Streams','Pandora Track Stations','Soundcloud Streams', 'TIDAL Popularity', 'ISRC']
df.drop(columns, axis=1, inplace=True)
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 3094 entries, 0 to 4599
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Track                   3094 non-null   object 
 1   Artist                  3094 non-null   object 
 2   Release Date            3094 non-null   object 
 3   All Time Rank           3094 non-null   object 
 4   Track Score             3094 non-null   float64
 5   Spotify Streams         3094 non-null   object 
 6   Spotify Playlist Count  3094 non-null   object 
 7   Spotify Playlist Reach  3094 non-null   object 
 8   Spotify Popularity      2917 non-null   float64
 9   YouTube Views           3094 non-null   object 
 10  YouTube Likes           3094 non-null   object 
 11  TikTok Posts            3094 non-null   object 
 12  TikTok Likes            3094 non-null   object 
 13  TikTok Views            3094 non-null   object 
 14  Shazam Counts           3094 non-null   objec

,Track,Artist,Release Date,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,TikTok Views,Shazam Counts,Explicit Track
0,MILLION DOLLAR BABY,Tommy Richman,4/26/2024,1,725.4,"390,470,936","30,716","196,631,588",92.0,"84,274,754","1,713,126","5,767,700","651,565,900","5,332,281,936","2,669,262",0
1,Not Like Us,Kendrick Lamar,5/4/2024,2,545.9,"323,703,884","28,113","174,597,137",92.0,"116,347,040","3,486,739","674,700","35,223,547","208,339,025","1,118,279",1
2,i like the way you kiss me,Artemas,3/19/2024,3,538.4,"601,309,283","54,331","211,607,669",92.0,"122,599,116","2,228,730","3,025,400","275,154,237","3,369,120,610","5,285,340",0
3,Flowers,Miley Cyrus,1/12/2023,4,444.9,"2,031,280,633","269,802","136,569,078",85.0,"1,096,100,899","10,629,796","7,189,811","1,078,757,968","14,603,725,994","11,822,942",0
5,Lovin On Me,Jack Harlow,11/10/2023,6,410.1,"670,665,438","105,892","175,421,034",83.0,"131,148,091","1,392,593","4,202,367","214,943,489","2,938,686,633","4,517,131",1
7,Gata Only,FloyyMenor,2/2/2024,8,375.8,"675,079,153","40,094","211,236,940",92.0,"228,382,568","1,439,495","3,500,000","338,546,668","3,804,584,163","6,063,523",1
9,BAND4BAND (feat. Lil Baby),Central Cee,5/23/2024,10,330.6,"90,676,573","10,400","184,199,419",86.0,"32,735,244","988,682","325,800","121,574,500","974,656,200","666,302",1
12,LUNCH,Billie Eilish,5/17/2024,13,316.3,"221,636,195","13,800","197,280,692",92.0,"40,022,524","1,307,290","195,300","45,889,000","360,017,000","450,344",0
14,bathroom floor,Kids With Buns,10/7/2022,15,301.6,"297,185",277,"24,445",12.0,"6,736,502,312","16,785,489",3,"1,088","22,234","3,288",0
15,LALA,Myke Towers,3/22/2023,16,299.9,"925,655,569","103,605","79,944,921",79.0,"934,302,031","4,294,166","11,054,500","521,725,116","7,499,234,052","2,765,808",1


In [4]:
##Track,Artist,Release Date,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,TikTok Views,Shazam Counts,Explicit Track

for i in num_norm:
    df[i] = df[i].replace(',', '', regex=True).astype(int)
#4/26/2024
df['Release Date'] = pd.to_datetime(df['Release Date'], format='%m/%d/%Y')
df['Release Date'] = df['Release Date'].astype('datetime64[ns]')

string_norm = ['Track','Artist']
for i in string_norm:
    df[i] = df[i].replace(',', '', regex=True).astype(str)



In [5]:
#remove songs that were realesed before tik was big in the US 
#this date is just my opion
df = df[df['Release Date'] >= pd.to_datetime('2016-09-01')]


In [6]:
df.info()
df.head(20)

<class 'pandas.core.frame.DataFrame'>
Index: 2689 entries, 0 to 4599
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Track                   2689 non-null   object        
 1   Artist                  2689 non-null   object        
 2   Release Date            2689 non-null   datetime64[ns]
 3   All Time Rank           2689 non-null   object        
 4   Track Score             2689 non-null   float64       
 5   Spotify Streams         2689 non-null   int64         
 6   Spotify Playlist Count  2689 non-null   object        
 7   Spotify Playlist Reach  2689 non-null   object        
 8   Spotify Popularity      2512 non-null   float64       
 9   YouTube Views           2689 non-null   int64         
 10  YouTube Likes           2689 non-null   int64         
 11  TikTok Posts            2689 non-null   int64         
 12  TikTok Likes            2689 non-null   int64        

,Track,Artist,Release Date,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,TikTok Views,Shazam Counts,Explicit Track
0,MILLION DOLLAR BABY,Tommy Richman,2024-04-26,1,725.4,390470936,"30,716","196,631,588",92.0,84274754,1713126,5767700,651565900,5332281936,2669262,0
1,Not Like Us,Kendrick Lamar,2024-05-04,2,545.9,323703884,"28,113","174,597,137",92.0,116347040,3486739,674700,35223547,208339025,1118279,1
2,i like the way you kiss me,Artemas,2024-03-19,3,538.4,601309283,"54,331","211,607,669",92.0,122599116,2228730,3025400,275154237,3369120610,5285340,0
3,Flowers,Miley Cyrus,2023-01-12,4,444.9,2031280633,"269,802","136,569,078",85.0,1096100899,10629796,7189811,1078757968,14603725994,11822942,0
5,Lovin On Me,Jack Harlow,2023-11-10,6,410.1,670665438,"105,892","175,421,034",83.0,131148091,1392593,4202367,214943489,2938686633,4517131,1
7,Gata Only,FloyyMenor,2024-02-02,8,375.8,675079153,"40,094","211,236,940",92.0,228382568,1439495,3500000,338546668,3804584163,6063523,1
9,BAND4BAND (feat. Lil Baby),Central Cee,2024-05-23,10,330.6,90676573,"10,400","184,199,419",86.0,32735244,988682,325800,121574500,974656200,666302,1
12,LUNCH,Billie Eilish,2024-05-17,13,316.3,221636195,"13,800","197,280,692",92.0,40022524,1307290,195300,45889000,360017000,450344,0
14,bathroom floor,Kids With Buns,2022-10-07,15,301.6,297185,277,"24,445",12.0,6736502312,16785489,3,1088,22234,3288,0
15,LALA,Myke Towers,2023-03-22,16,299.9,925655569,"103,605","79,944,921",79.0,934302031,4294166,11054500,521725116,7499234052,2765808,1


Module 4

In [ ]:
'Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,Spotify Popularity,YouTube Views,YouTube Likes,TikTok Posts,TikTok Likes,TikTok Views,YouTube Playlist Reach,Apple Music Playlist Count,AirPlay Spins,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,Explicit Track'

num_col = ['Track Score','YouTube Views','YouTube Likes','TikTok Posts','TikTok Likes','TikTok Views','Shazam Counts','Explicit Track']

In [ ]:
df_mod4 = df.copy()
for x in num_col + ['Spotify Streams']:
    df_mod4 = df_mod4.dropna(subset=[x])

In [ ]:
for i in num_col + ['Spotify Streams']:
    df_mod4[i] = df_mod4[i].replace(',', '', regex=True).astype(int)

In [ ]:
plt.title('tests')
plt.plot(df_mod4['Spotify Streams'], df_mod4['TikTok Views'], 'x')
plt.xlabel('Spotify Streams')
plt.ylabel('TikTok Views')
plt.xlim(0, 5000000000)   
plt.ylim(0, 25000000000)

In [ ]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_mod4[num_col])

In [ ]:
inertia = []
k_range = range(1, 50)
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(scaled_data)
    inertia.append(kmeans.inertia_)


In [ ]:
def log_func(x, a, b):
    return a * np.log(x) + b 

params, _ = curve_fit(log_func, k_range[1:], inertia[1:])
log_fit = log_func(np.array(k_range[1:]), *params)

In [ ]:

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o', color='palegreen')
plt.plot(k_range[1:], log_fit, 'r--', color = 'fuchsia')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

In [ ]:

k = 5
kmeans = KMeans(n_clusters=k, random_state=42)
df['Cluster'] = kmeans.fit_predict(scaled_data)

pca = PCA(n_components=2)
reduced = pca.fit_transform(scaled_data)
df['PCA1'] = reduced[:, 0]
df['PCA2'] = reduced[:, 1]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='PCA1', y='PCA2', hue='Cluster', palette='tab10')
plt.title('2D PCA View of Clusters')
plt.show()

cluster_analysis = df.groupby('Cluster')['Spotify Streams'].describe()
print(cluster_analysis)

for i in range(k):
    print(f"\nCluster {i} top tracks by streams:")
    print(df[df['Cluster'] == i][['Track', 'Artist', 'Spotify Streams']].sort_values(by='Spotify Streams', ascending=False).head(3))


Module 6

In [7]:
#tiktok_features = ['TikTok Views', 'TikTok Likes', 'TikTok Posts']
features = ['Track Score','YouTube Views','YouTube Likes','TikTok Posts','TikTok Likes','TikTok Views','Shazam Counts']
df['is_hit'] = df['Spotify Streams'].apply(lambda x: 1 if x >= 300_000_000 else 0)

In [9]:
X = df[features]
y = df['Spotify Streams']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#model = RandomForestClassifier(n_estimators=100, random_state=42)
#LogisticRegression LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

y_test_array = np.array(y_test).reshape(-1,1)
y_pred_array = np.array(y_pred)

model.fit(y_test_array, y_pred_array)
trend_line = model.predict(y_test_array)


'y_pred = model.predict(X_test)\nprint("Classification Report:\n", classification_report(y_test, y_pred))\nprint("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))'

In [ ]:
'''importances = model.feature_importances_
sns.barplot(x=importances, y=tiktok_features)
plt.title("Importance of TikTok Features for Predicting Hits")
plt.show()'''

In [11]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("Mean Squared Error:", mse)
print("R^2 Score:", r2)

Mean Squared Error: 9.110726446282466e+16
R^2 Score: 0.5799226088721101


In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test, y_pred, alpha=0.5, color='blue')
plt.plot(y_test, trend_line, color='red', linewidth=2)
plt.xlabel("True Labels")
plt.ylabel("Predicted Labels")
plt.title("True vs Predicted Labels")
plt.grid(True)
plt.xlim(0, 5000000000)   
plt.ylim(0, 5000000000)
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test, y_pred, alpha=0.5, color='blue')
plt.plot(y_test, trend_line, color='red', linewidth=2)
plt.xlabel("True Labels")
plt.ylabel("Predicted Labels")
plt.title("True vs Predicted Labels")
plt.grid(True)
plt.xlim(0, 2000000000)   
plt.ylim(0, 2000000000)
plt.show()